In [1]:
from keras.layers import Input, Dense, Conv1D, Flatten, MaxPooling1D, Dropout
from keras.models import Model, Sequential

def network(input_window_size=60, filter_number=32, conv_window=(3,), pooling_window=(2,), dropout_rate=[],
            activation='relu', dense_activation='softmax', optimizer='adam', loss='categorical_crossentropy', layers=1):
    model = Sequential()
    
    # Input Layer
    model.add(Conv1D(filter_number, conv_window, activation=activation, padding='same', input_shape=(input_window_size, 1)))
    model.add(MaxPooling1D(pooling_window, padding='same'))
    model.add(Dropout(dropout_rate[0]))

    
    # Hidden Layers
    current_filter = 1
    filter_number_temp = filter_number
    for i in range(layers):
        filter_number_temp = filter_number_temp *  2
        model.add(Conv1D(filter_number_temp, conv_window, activation=activation, padding='same'))
        model.add(MaxPooling1D(pooling_window, padding='same'))
        model.add(Dropout(dropout_rate[current_filter]))
        current_filter = current_filter + 1
    
    # Output Layer
    model.add(Flatten())
    model.add(Dense(2, activation=dense_activation))

    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy', 'categorical_accuracy'])

    return model

Using TensorFlow backend.


In [2]:
import os
import h5py
import numpy as np
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split


def rolling_window(a, window, step_size, padding=True, copy=True):
    if copy:
        result = a.copy()
    else:
        result = a
    if padding:
        result = np.hstack((result, np.zeros(window)))
    shape = result.shape[:-1] + (result.shape[-1] - window + 1 - step_size, window)
    strides = result.strides + (result.strides[-1] * step_size,)
    return np.lib.stride_tricks.as_strided(result, shape=shape, strides=strides)


size = 60

base_path = '/home/irene/Documents/Datos-Tesis/Training_window_station_name'

total_files = len(os.listdir(base_path))
x_dataset = np.zeros(shape=(0, size))
y_dataset = np.zeros(shape=(0, 1))
count = 1
skipped = []
for file_path in os.listdir(base_path):
    current_index = 0
    
    h5f = h5py.File(os.path.join(base_path, file_path), 'r')
    
    x = rolling_window(h5f['normal'][:], size, 1)
    y = rolling_window(h5f['transformed'][:], size, 1)
    y = [[np.amax(array)] for array in y]

    x_dataset = np.vstack((x_dataset, x))
    y_dataset = np.vstack((y_dataset, y))
    
    print(count)
    count = count + 1

x_dataset = np.reshape(x_dataset, (len(x_dataset), size, 1))
y_dataset = np.reshape(y_dataset, (len(y_dataset), 1))
y_dataset = to_categorical(y_dataset)

indices = np.arange(x_dataset.shape[0])
np.random.shuffle(indices)
x_dataset= x_dataset[indices] 
y_dataset= y_dataset[indices]

x_train, x_test, y_train, y_test = train_test_split(x_dataset, y_dataset)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint, uniform
from keras import optimizers
from keras.wrappers.scikit_learn import KerasClassifier

# Convert to scikit-learn model (build_fn is a function that returns a model)
model = KerasClassifier(build_fn=network)

# list of parameters
input_window_size = [60]
layers = [1,2,3,4]
filter_number = sp_randint(4, 32)
conv_window = [(3,), (5,)]
pooling_window = [(2,), (4,)]
activation = ['relu']
dense_activation = ['softmax']
optimizer = ['adam']
loss = ['categorical_crossentropy']
dropout_rate = [uniform.rvs(0.0, 0.25, size = 5),
                uniform.rvs(0.0, 0.5, size = 5),
                uniform.rvs(0.0, 0.15, size = 5),
                uniform.rvs(0.0, 0.75, size = 5),
                uniform.rvs(0.0, 0.015, size = 5),
                uniform.rvs(0.0, 0.075, size = 5)]

epochs= sp_randint(30, 60)
batch_size = sp_randint(32, 128)

param_dist = dict(input_window_size=input_window_size,
                  filter_number=filter_number, 
                  conv_window=conv_window, 
                  pooling_window=pooling_window,
                  dropout_rate = dropout_rate,
                  activation=activation, 
                  dense_activation=dense_activation, 
                  optimizer=optimizer, 
                  loss=loss, 
                  layers=layers,
                  epochs=epochs,
                  batch_size=batch_size)

# Random Search
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, cv=3, verbose=2, n_iter=64)
result = random_search.fit(x_train, y_train)


Fitting 3 folds for each of 64 candidates, totalling 192 fits
[CV] activation=relu, batch_size=82, conv_window=(5,), dense_activation=softmax, dropout_rate=[0.4860259  0.02084982 0.4743487  0.05438419 0.41047941], epochs=54, filter_number=21, input_window_size=60, layers=1, loss=categorical_crossentropy, optimizer=adam, pooling_window=(4,) 
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Instructions for updating:
Use tf.cast instead.
Epoch 1/54
76076/76076 [==============================] - 8s 99us/step - loss: 0.2234 - acc: 0.9410 - categorical_accuracy: 0.9410
Epoch 2/54
76076/76076 [==============================] - 7s 92us/step - loss: 0.1992 - acc: 0.9429 - categorical_accuracy: 0.9429
Epoch 3/54
76076/76076 [==============================] - 7s 90us/step - loss: 0.1878 - acc: 0.9457 - categorical_accuracy: 0.9457
Epoch 4/54
76076/76076 [==============================] - 7s 94us/step - loss: 0.1820 - acc: 0.9476 - categorical_accuracy: 0.9476
Epoch 5/54
76076/76076 [==============================] - 7s 93us/step - loss: 0.1774 - acc: 0.9490 - categorical_accuracy: 0.9490
Epoch 6/54
76076/76076 [==============================] - 7s 92us/step - loss: 0.1748 - acc: 0.9501 - categorical_accuracy: 0.9501
Epoch 7/54
76076/76076 [==============================] - 7s 92us/step - loss: 0.1719 - acc: 0.9513 - categorical_accuracy: 0.9513
Epoch 8/54
76076/76076 [===========

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  6.6min remaining:    0.0s


Epoch 1/54
76076/76076 [==============================] - 7s 91us/step - loss: 0.2277 - acc: 0.9384 - categorical_accuracy: 0.9384
Epoch 2/54
76076/76076 [==============================] - 7s 87us/step - loss: 0.2051 - acc: 0.9418 - categorical_accuracy: 0.9418
Epoch 3/54
76076/76076 [==============================] - 7s 93us/step - loss: 0.1960 - acc: 0.9435 - categorical_accuracy: 0.9435
Epoch 4/54
76076/76076 [==============================] - 7s 90us/step - loss: 0.1854 - acc: 0.9460 - categorical_accuracy: 0.9460
Epoch 5/54
76076/76076 [==============================] - 7s 91us/step - loss: 0.1795 - acc: 0.9477 - categorical_accuracy: 0.9477
Epoch 6/54
76076/76076 [==============================] - 7s 92us/step - loss: 0.1768 - acc: 0.9486 - categorical_accuracy: 0.9486
Epoch 7/54
76076/76076 [==============================] - 6s 84us/step - loss: 0.1746 - acc: 0.9492 - categorical_accuracy: 0.9492
Epoch 8/54
76076/76076 [==============================] - 7s 95us/step - loss: 0.17

...

76076/76076 [==============================] - 29s 379us/step - loss: 0.0197 - acc: 0.9943 - categorical_accuracy: 0.9943
Epoch 46/58
76076/76076 [==============================] - 25s 331us/step - loss: 0.0199 - acc: 0.9943 - categorical_accuracy: 0.9943
Epoch 47/58
  738/76076 [..............................] - ETA: 2:31 - loss: 0.0296 - acc: 0.9892 - categorical_accuracy: 0.9892

/home/irene/virtualenvs/obspy/lib/python3.6/site-packages/keras/callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.186475). Check your callbacks.
  % delta_t_median)


76076/76076 [==============================] - 25s 330us/step - loss: 0.0213 - acc: 0.9935 - categorical_accuracy: 0.9935
Epoch 48/58
76076/76076 [==============================] - 27s 349us/step - loss: 0.0197 - acc: 0.9939 - categorical_accuracy: 0.9939
Epoch 49/58
76076/76076 [==============================] - 26s 348us/step - loss: 0.0181 - acc: 0.9943 - categorical_accuracy: 0.9943
Epoch 50/58
76076/76076 [==============================] - 25s 325us/step - loss: 0.0211 - acc: 0.9939 - categorical_accuracy: 0.9939
Epoch 51/58
76076/76076 [==============================] - 25s 324us/step - loss: 0.0189 - acc: 0.9946 - categorical_accuracy: 0.9946
Epoch 52/58
 1722/76076 [..............................] - ETA: 4:07:43 - loss: 0.0213 - acc: 0.9930 - categorical_accuracy: 0.9930- ETA: 7:46:09 - loss: 0.0197 - acc: 0.9878 - categorical_accuracy: 0.9878 - ETA: 7:33:26 - loss: 0.0167 - acc: 0.9919 - categorical_accuracy: 0.991 - ETA: 7:16:18 - loss: 0.0139 - acc: 0.9939 - categorical_accu

/home/irene/virtualenvs/obspy/lib/python3.6/site-packages/keras/callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (4.175285). Check your callbacks.
  % delta_t_median)
/home/irene/virtualenvs/obspy/lib/python3.6/site-packages/keras/callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (2.533253). Check your callbacks.
  % delta_t_median)
/home/irene/virtualenvs/obspy/lib/python3.6/site-packages/keras/callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (2.268135). Check your callbacks.
  % delta_t_median)
/home/irene/virtualenvs/obspy/lib/python3.6/site-packages/keras/callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (1.183686). Check your callbacks.
  % delta_t_median)


76076/76076 [==============================] - 368s 5ms/step - loss: 0.0195 - acc: 0.9941 - categorical_accuracy: 0.9941
Epoch 53/58
76076/76076 [==============================] - 27s 360us/step - loss: 0.0195 - acc: 0.9944 - categorical_accuracy: 0.9944
Epoch 54/58
76076/76076 [==============================] - 27s 349us/step - loss: 0.0206 - acc: 0.9938 - categorical_accuracy: 0.9938
Epoch 55/58
76076/76076 [==============================] - 25s 325us/step - loss: 0.0163 - acc: 0.9952 - categorical_accuracy: 0.9952
Epoch 56/58
76076/76076 [==============================] - 25s 334us/step - loss: 0.0165 - acc: 0.9951 - categorical_accuracy: 0.9951
Epoch 57/58
76076/76076 [==============================] - 26s 348us/step - loss: 0.0184 - acc: 0.9942 - categorical_accuracy: 0.9942
Epoch 58/58
76076/76076 [==============================] - 12s 156us/step
[CV]  activation=relu, batch_size=123, conv_window=(5,), dense_activation=softmax, dropout_rate=[0.08809469 0.04242648 0.00138052 0.043

In [ ]:
# Show the results
print("Best: %f using %s" % (result.best_score_, result.best_params_))
means = result.cv_results_['mean_test_score']
stds = result.cv_results_['std_test_score']
params = result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
   print("%f (%f) with: %r" % (mean, stdev, param))